# Implementation of face recognition using neural net 

In [1]:
%matplotlib inline

In [2]:
import cv2
import numpy as np
import os
import dlib
from skimage import io
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

Using TensorFlow backend.


#### Listing the path of all the images

In [4]:
DatasetPath = []
for i in os.listdir("Feret"):
    DatasetPath.append(os.path.join("Feret", i))

#### Reading each image and assigning respective labels

In [5]:
imageData = []
imageLabels = []

In [6]:
for i in DatasetPath:
#     imgRead = cv2.imread(i)
#     imageData.append(imgRead)
    
    labelRead = int(os.path.split(i)[1].split("_")[0]) - 1
    imageLabels.append(labelRead)

#### Preprocessing: Face Detection using OpenCV and cropping the image to a size of 150 * 150

In [7]:
faceDetectClassifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [8]:
imageDataFin = []
for file_name in DatasetPath:
    gray = cv2.imread(file_name,0)

    faces = faceDetectClassifier.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )
    faces = faceDetectClassifier.detectMultiScale(gray, 1.2, 5)

#     print("Found {0} faces!".format(len(faces)))

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(gray, (x, y), (x+w, y+h), (0, 255, 0), 2)
        thing = gray[y: y + w, x: x + h]

    thing = cv2.resize(thing,(150,150))
    imageDataFin.append(thing)

In [9]:
c = np.array(imageDataFin)

In [10]:
c.shape

(48, 150, 150)

In [11]:
len(imageLabels)

48

#### Splitting Dataset into train and test

In [12]:
X_train, X_test, y_train, y_test = train_test_split(np.array(imageDataFin),np.array(imageLabels), train_size=0.9, random_state = 20)

In [13]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [14]:
X_train.shape

(43, 150, 150)

In [15]:
X_test.shape

(5, 150, 150)

In [16]:
nb_classes = 15
y_train = np.array(y_train) 
y_test = np.array(y_test) 

In [17]:
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

#### Converting each 2d image into 1D vector

In [18]:
X_train = X_train.reshape(43, 150*150)
X_test = X_test.reshape(5, 150*150)

In [19]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

In [20]:
print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

Training matrix shape (43, 22500)
Testing matrix shape (5, 22500)


#### Defining the model

In [21]:
model = Sequential()
model.add(Dense(512,input_shape=(X_train.shape[1],)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               11520512  
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 15)                7695      
__________

In [23]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

#### Training

In [24]:
model.fit(X_train, Y_train, batch_size=64, nb_epoch=50, verbose=1, validation_data=(X_test, Y_test))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 43 samples, validate on 5 samples
Epoch 1/50
43/43 [==============================] - 0s - loss: 2.6918 - acc: 0.0930 - val_loss: 11.5178 - val_acc: 0.0000e+00
Epoch 2/50
43/43 [==============================] - 0s - loss: 10.8685 - acc: 0.1860 - val_loss: 9.6709 - val_acc: 0.4000
Epoch 3/50
43/43 [==============================] - 0s - loss: 12.9024 - acc: 0.1860 - val_loss: 9.6723 - val_acc: 0.4000
Epoch 4/50
43/43 [==============================] - 0s - loss: 12.1411 - acc: 0.1860 - val_loss: 9.6734 - val_acc: 0.4000
Epoch 5/50
43/43 [==============================] - 0s - loss: 12.2762 - acc: 0.1163 - val_loss: 9.6709 - val_acc: 0.4000
Epoch 6/50
43/43 [==============================] - 0s - loss: 12.3801 - acc: 0.2093 - val_loss: 9.6709 - val_acc: 0.4000
Epoch 7/50
43/43 [==============================] - 0s - loss: 12.8937 - acc: 0.1395 - val_loss: 9.6709 - val_acc: 0.4000
Epoch 8/50
43/43 [==============================] - 0s - loss: 12.8233 - acc: 0.1628 - val_loss: 9.

#### Evaluating the performance

In [25]:
loss, accuracy = model.evaluate(X_test,Y_test, verbose=0)

In [26]:
loss

9.6708574295043945

In [27]:
accuracy

0.40000000596046448

In [28]:
predicted_classes = model.predict_classes(X_test)

5/5 [==============================] - 0s


In [29]:
correct_classified_indices = np.nonzero(predicted_classes == y_test)[0]
incorrect_classified_indices = np.nonzero(predicted_classes != y_test)[0]

In [30]:
correct_classified_indices

array([0, 1])

In [31]:
incorrect_classified_indices

array([2, 3, 4])